## Hybrid Neural-Net GARCH Models

### 1. NNGARCH

### 2. Recurrent Neural Net & GARCH

#### 2-1. RNN-GARCH

#### 2-2. $\alpha$RNN-GARCH

#### 2-3. $\alpha_t$RNN-GARCH

### 3. Market Long Memories

#### 3-1. FIRNN-GARCH

#### 3-2. $\alpha$FIRNN-GARCH

#### 3-3. $\alpha_t$FIRNN-GARCH